In [1]:
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt

### 1. Object tracking

In [2]:
## 동영상
cap = cv2.VideoCapture("./fig/video/PETS2000.avi")

if not cap.isOpened():
    print("Video open failed")
    sys.exit()

while True:
    ret, frame = cap.read()

    cv2.imshow("frame", frame)

    if cv2.waitKey(20) == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# cv2.connectedComponentsWithStats(image, labels=None, stats=None, centroids=None, connectivity=None, ltype=None) -> retval, labels, stats, centroids

#  image: 8비트 1채널 영상
#  labels: 레이블 맵 행렬. 입력 영상과 같은 크기. numpy.ndarray.
#  stats: 각 객체의 바운딩 박스, 픽셀 개수 정보를 담은 행렬. numpy.ndarray. shape=(N, 5), dtype=numpy.int32.
#  centroids: 각 객체의 무게 중심 위치 정보를 담은 행렬 numpy.ndarray. shape=(N, 2), dtype=numpy.float64.
# ltype: labels 행렬 타입. cv2.CV_32S 또는 cv2.CV_16S. 기본값은 cv2.CV_32S

In [3]:
cap = cv2.VideoCapture("./fig/video/PETS2000.avi")
if not cap.isOpened():
    print("Video open failed")
    sys.exit()

ret, background = cap.read()
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
background_gray_G = cv2.GaussianBlur(background_gray, (0, 0), 1.) 
# cv2.imshow("background", background)
# cv2.waitKey()
while True:
    ret, frame = cap.read() # fps: frame per second

    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray_G = cv2.GaussianBlur(frame_gray, (0, 0), 1.)
    diff_G = cv2.absdiff(frame_gray_G, background_gray_G)
    ret_g, mask_g = cv2.threshold(diff_G, 50, 255, cv2.THRESH_BINARY)
    # mask_g = cv2.dilate(mask_g, None, iterations=1)
    # mask_g = cv2.morphologyEx(mask_g, cv2.MORPH_CLOSE, None, iterations=2)
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_g)
    for i in range(1, cnts):
        x, y, w, h, area = stats[i]
        if area <= 400:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow("frame", frame)
    cv2.imshow("diff", diff_G)
    cv2.imshow("mask_g", mask_g)

    if cv2.waitKey(20) == 27:
        break

cv2.destroyAllWindows()
cap.release()


### 2. 이동 평균 배경 차분

In [ ]:
# accumulateWeighted(src, dst, alpha, mask) -> dst
# src: 입력영상
# dis: 출력영상 (32bit, 64bit)
# alpha : 축적가중치
# mask: 마스트 영상

In [4]:
## 이동 평균 배경 차분
cap = cv2.VideoCapture("./fig/video/PETS2000.avi")

if not cap.isOpened():
    print("Video open failed")
    sys.exit()
ret, back = cap.read()
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0, 0), 1.)
fback = back.astype(np.float32)
while True:
    ret, frame = cap.read()
    if not ret:
        print("frame is None")
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (0, 0), 1)
    cv2.accumulateWeighted(frame_gray, fback, 0.01)
    back = fback.astype(np.uint8)
    diff = cv2.absdiff(frame_gray, back)
    ret, mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
    for i in range(1, cnts):
        x, y, w, h, area = stats[i]
        if area < 100:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    cv2.imshow("back",back)
    
    if cv2.waitKey(20) == ord("q"):
        break

cv2.destroyAllWindows()
cap.release()

### 3. Mixture of Gaussian

In [ ]:
# createBackgroundSubtractorMOG2([, history, varThreshold, detectShadows) -> retval
# history: 배경 업데이트 과거 길이, 기본 500
# varThreshold: 새로 들어온 픽셀값이 배경 모델에 의해서 잘 표현되는지를 판단, 기본값 16
# detectShadows: 그림자 검출 여부

# cv2.createBackgroundSubtractor.apply(image, fgmask = None, learningRate=Nome) -> fgmask
# image: 입력영상
# fgmask: None
# learningRate: 기본값 -1

In [44]:
## Mixture of Gaussian
cap = cv2.VideoCapture("./video/PETS2000.avi")

if not cap.isOpened():
    print("Video load failed")
    sys.exit()

bs = cv2.createBackgroundSubtractorMOG2()
while True:
    ret, frame = cap.read()
    # frame = cv2.GaussianBlur(frame, (0, 0), 1)
    # frame_m = cv2.medianBlur(frame, 3)
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fgmask = bs.apply(gray)
    back = bs.getBackgroundImage()
    cnts, labels, stats, centroids = cv2.connectedComponentsWithStats(fgmask)
    for i in range(1, cnts):
        x, y, w, h, area = stats[i]
        if area <= 400:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)

    cv2.imshow("frame", frame)
    cv2.imshow("background", back)
    cv2.imshow("fgmask", fgmask)
    if cv2.waitKey(20) ==  27:
        break

cv2.destroyAllWindows()
cap.release()


### 4. 기하학적 모멘트 

In [5]:
## 기하학적 모멘트 (Hu 불변 모멘트)
obj = cv2.imread("./fig/spades.png", cv2.IMREAD_GRAYSCALE)
src = cv2.imread("./fig/symbols.png", cv2.IMREAD_GRAYSCALE)
_, obj_bin = cv2.threshold(obj, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
obj_pts = obj_contours[0]
_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
src_contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for pts in src_contours:
    if cv2.contourArea(pts) < 1000:
        continue
    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255, 0, 0), 1)
    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)
    cv2.putText(dst, str(round(dist, 3)), (rc[0], rc[1] - 3), cv2.FONT_HERSHEY_COMPLEX, 0.8,
                (255, 0, 0), 1, cv2.LINE_AA)
    
    if dist < 0.1:
        cv2.rectangle(dst, rc, (0, 0, 255), 2)
        cv2.putText(dst, str(round(dist, 3)), (rc[0], rc[1] - 3), cv2.FONT_HERSHEY_COMPLEX, 0.8,
                (0, 0, 255), 2, cv2.LINE_AA)
        
cv2.imshow("obj", obj)
cv2.imshow("src", src)
cv2.imshow("dst", dst)
# cv2.imshow("obj_bin", obj_bin)

cv2.waitKey()
cv2.destroyAllWindows()